# **Tarefa 1 – Web Scraping em Ambiente Controlado (50%)**
### **Disciplina**: Coleta, Preparação e Análise de Dados - Prof. Lucas Pessutto

#### **Integrantes do grupo**: Ana Cristina Schmidt e Jeniffer Borges

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re 
import time
from datetime import datetime
import csv
import os

In [2]:
os.makedirs('html_dos_sites', exist_ok=True)
os.makedirs('dados_em_csv', exist_ok=True)

url_base = 'http://127.0.0.1:8000/places/default/index/'

url_pais = 'http://127.0.0.1:8000'

In [3]:
def extrair_dados_pais(pais_link):
    response = requests.get(pais_link)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        pais = soup.find('tr', id='places_country__row')
        pais_nome = pais.find('td', class_='w2p_fw').text.strip() if pais else "Nao encontrado"

        arq_html(pais_nome, response.text)
        
        capital = soup.find('tr', id='places_capital__row')
        capital_nome = capital.find('td', class_='w2p_fw').text.strip() if capital else "Nao encontrado"

        currency = soup.find('tr', id='places_currency_name__row')
        currency_nome = currency.find('td', class_='w2p_fw').text.strip() if currency else "Nao encontrado"

        continent = soup.find('tr', id='places_continent__row')
        continent_nome = continent.find('td', class_='w2p_fw').text.strip() if continent else "Nao encontrado"

        continentes = {
            "AF": "Africa",
            "AN": "Antartida",
            "AS": "Asia",
            "EU": "Europa",
            "NA": "America do Norte",
            "OC": "Oceania",
            "SA": "America do Sul"
        }

        continent_nome = continentes.get(continent_nome, continent_nome)
        
        return {
            'pais': pais_nome,
            'capital': capital_nome,
            'moeda': currency_nome,
            'continente': continent_nome,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
    else:
        print(f"Erro ao acessar a pagina do pais {pais_link} ({response.status_code})")
        return None

In [4]:
def extrair_link_pais(soup):
    paises_links = []
    linhas = soup.select('table tr')
    
    for linha in linhas:
        colunas = linha.find_all('td')
        for coluna in colunas:
            link_tag = coluna.find('a')
            if link_tag:
                pais_nome = link_tag.text.strip()
                pais_link = url_pais + link_tag['href']
                paises_links.append((pais_nome, pais_link))
    return paises_links

In [5]:
def arq_csv(dados, nome_arquivo='dados_em_csv/dados_paises.csv'):
    arquivo_existe = os.path.isfile(nome_arquivo)

    with open(nome_arquivo, mode='a', newline='', encoding='utf-8') as arquivo:
        write = csv.DictWriter(arquivo, fieldnames=['pais', 'capital', 'moeda', 'continente', 'timestamp'])

        if not arquivo_existe:
            write.writeheader()

        write.writerow(dados)

In [6]:
def arq_html(nome_pais, conteudo_html):
    nome_arquivo = f"html_dos_sites/{nome_pais.replace(' ', '_')}.html"
    
    with open(nome_arquivo, 'w', encoding='utf-8') as arquivo:
        arquivo.write(conteudo_html)
    print(f"A pagina HTML do pais {nome_pais} foi salva")

In [7]:
def ler_csv_sem_timestamp(nome_arquivo='dados_em_csv/dados_paises.csv'):
    if os.path.isfile(nome_arquivo):
        dados = pd.read_csv(nome_arquivo, usecols=['pais', 'capital', 'moeda', 'continente'])
        return dados
    else:
        print(f"O arquivo {nome_arquivo} não foi encontrado.")
        return pd.DataFrame(columns=['pais', 'capital', 'moeda', 'continente'])

In [8]:
def verificar_e_atualizar_csv(pais_dados, nome_arquivo='dados_em_csv/dados_paises.csv'):
    dados_atuais = ler_csv_sem_timestamp(nome_arquivo)
    dados_pais_atual = dados_atuais[dados_atuais['pais'] == pais_dados['pais']]
    
    if dados_pais_atual.empty or not dados_pais_atual.iloc[0].equals(pd.Series(pais_dados).drop('timestamp')):
        print(f"Atualizando dados de {pais_dados['pais']} no CSV.")
        arq_csv(pais_dados, nome_arquivo)
    else:
        print(f"Nenhuma alteração de CSV necessaria para {pais_dados['pais']}.")

In [9]:
def crawler(n_pgs=25, nome_arquivo='dados_em_csv/dados_paises.csv'):
    pg_atual = 0

    while pg_atual <= n_pgs:
        print(f"Processando a pagina {pg_atual}...")

        url = url_base + str(pg_atual)
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Erro ao acessar a pagina {pg_atual}")
            break

        soup = BeautifulSoup(response.text, 'html.parser')

        paises_links = extrair_link_pais(soup)

        for pais_nome, pais_link in paises_links:
            pais_dados = extrair_dados_pais(pais_link)
            if pais_dados:
                verificar_e_atualizar_csv(pais_dados, nome_arquivo)

        pg_atual += 1
        time.sleep(1)

In [10]:
crawler()

Processando a pagina 0...
A pagina HTML do pais Afghanistan foi salva
Nenhuma alteração de CSV necessaria para Afghanistan.
A pagina HTML do pais Aland Islands foi salva
Nenhuma alteração de CSV necessaria para Aland Islands.
A pagina HTML do pais Albania foi salva
Nenhuma alteração de CSV necessaria para Albania.
A pagina HTML do pais Algeria foi salva
Nenhuma alteração de CSV necessaria para Algeria.
A pagina HTML do pais American Samoa foi salva
Nenhuma alteração de CSV necessaria para American Samoa.
A pagina HTML do pais Andorra foi salva
Nenhuma alteração de CSV necessaria para Andorra.
A pagina HTML do pais Angola foi salva
Nenhuma alteração de CSV necessaria para Angola.
A pagina HTML do pais Anguilla foi salva
Nenhuma alteração de CSV necessaria para Anguilla.
A pagina HTML do pais Antarctica foi salva
Atualizando dados de Antarctica no CSV.
A pagina HTML do pais Antigua and Barbuda foi salva
Nenhuma alteração de CSV necessaria para Antigua and Barbuda.
Processando a pagina 1.